In [ ]:
%load_ext autoreload
%autoreload 2
from fedflax import train
from models import LeNet
from data import fetch_data
from utils import angle_err, opt_create, return_l2
import jax
from jax import numpy as jnp
from flax import nnx
from tqdm.auto import tqdm
from functools import reduce

In [ ]:
model = LeNet(jax.random.key(42))
ds_train = fetch_data("feature", beta=1.)
ds_test = fetch_data("feature", partition="test", beta=1.)
ds_val = fetch_data("feature", partition="val", beta=1.)

# Measurement of functional interference between one client's update vs all updates
entanglement_fn = nnx.jit(nnx.vmap(
    lambda model_g, models, x, z: jnp.abs(model_g(x, z) - models(x, z)), 
    in_axes=(None, 0, 0, 0)
))

In [ ]:
distances = [...]
fl_entanglement_scores = []
rand_entanglement_scores = []
for distance in distances:
    # Get federated models which are this far apart
    fl_models = ...
    model_g = ...
    fl_entanglement_scores.append(reduce(
        lambda acc, batch: acc + entanglement_fn(model_g, fl_models, *batch[1:]).mean(), 
        ds_test,
        0.
    ) / len(ds_test))
    # Now check if this disentanglement is better than random
    rand_models = ...
    rstate, rparams, rrest = nnx.split(rand_models, (nnx.Param, nnx.BatchStat), ...)
    rparams = jax.tree.map(lambda p: p.mean(0), rparams)
    rand_model_g = nnx.merge(rstate, rparams, rrest)
    rand_entanglement_scores.append(reduce(
        lambda acc, batch: acc + entanglement_fn(model, rand_models, *batch[1:]).mean(), 
        ds_test,
        0.
    ) / len(ds_test))